## Introduction:

Neutrino physicists are interested determining the properties of neutrinos that interact within our detector. These include, for example, the energy, direction, and flavor of the neutrino. In IceCube, the most common flavors of neutrinos are electron neutrions ($\nu_e$) and muon neutrinos ($\nu_\mu$). Knowing the flavor of neutrino helps us learn more about how these cosmological neutrinos are produced. We can tell the flavor of the neutrino by the type of particle it produces: electron neutrinos produce electrons, and muon neutrinos produce muons.

Below you can see an example of a **track** from a muon (left) and a **cascade** from an electron (right) in IceCube. The spheres show the sensors of the detector that have observed light. Larger spheres mean that more photons were observed, and the color indicates the time that photons hit the sensor, where red is earlier and green is later. We can use the different shapes to tell which type of neutrino hit the detector.

![](../resources/images/track_cascade.png)

In this activity, we will try to distinguish between electrons and muons in IceCube using artificial intelligence. We will train a neural network similar to the one depicted below using images of tracks and cascades in IceCube. You can tweak the parameters of the network to try to get the best performance possible. Then, you can see how your own guesses for the neutrino flavor compare to those of the network!

<img src="../resources/images/neural_network.png" alt="nn" width="400"/>



First, we will need to download the data from google drive and import some Python code that will help run this exercise. Assuming you are using Google Colab, the files will not download to your own computer. Run the cell below by clicking the button in the top left corner, or by pressing shift+enter.

In [ ]:
# needed for figures to appear in colab 
from google.colab import output
output.enable_custom_widget_manager()

!rm -rf sample_data

# download reduced_electrons.parquet
!gdown https://drive.google.com/uc?id=1BVTkqniSfwJsVsUnNhM_FjMueXfotr2f

# download reduced_muons.parquet
!gdown https://drive.google.com/uc?id=1nJpyojI8CAwaq5P_ZcqgeKhfBEVCQn9n

# download pre-prepared analysis code
!rm -rf IceCube_MasterClass_at_Harvard2024
!git clone "https://github.com/kcarloni/IceCube_MasterClass_at_Harvard2024";

# Import some standard python libraries
import sys
import numpy as np
import matplotlib.pyplot as plt

# Import some custom libraries for this example
sys.path.insert(0, "./IceCube_MasterClass_at_Harvard2024/")
from src.ml_tools import *

The next cell takes the datasets that we just downloadaed and turns them into a format that we can use to train the neural network. Then, we create the network and the training dataest. There are two variables that you can play with here:
- "width": this determines the width of the hidden layer of the network. A larger width means that the network will have more parameters
- "N_train": this determines the number of IceCube images to use to train the network. More training data will *generally* lead to better performance

In [ ]:
# Load a Python class that will guide this ML example
ML_Helper = MLHelper("reduced_muons.parquet",
                     "reduced_electrons.parquet")

width = 10 # how "wide" the network is, which will determine the number of paremeters
N_train = 1000 # how many training examples to use. Pick a number between 1 and 4999

# Make the network
ML_Helper.MakeNetwork(width=width)

# Make the training data
ML_Helper.MakeTrainingDataset(N_train=N_train)

Now let's train the network! We will pass through the training data a certain number of times and adjust the parameters of the neural network as we go. We are using the binary cross entropy (BCE) loss function, which is the standard choice when trying to classify data into two different categories. There is a single variable you can play with here:
- "num_epochs": the number of times to pass through the training data. More epochs will *generally* lead to better performance

In [ ]:
# Let's train the network using the binary cross entropy loss function
num_epochs = 2 # decide how many times you want to pass through the training data
loss_dict = ML_Helper.train(num_epochs=num_epochs)
for epoch,losses in loss_dict.items():
    if epoch>0:
        plt.plot(epoch*len(losses)-1+np.arange(len(losses)+1),np.array([loss_dict[epoch-1][-1]]+losses),label=epoch)
    else:
        plt.plot(epoch*len(losses)+np.arange(len(losses)),np.array(losses),label=epoch)
plt.semilogy()
plt.ylabel("BCE Loss",fontsize=14)
plt.xlabel("Training Step",fontsize=14)
l = plt.legend(fontsize=14)
l.set_title("Epoch")
plt.show()

Now that we've trained the network, let's have some fun with it! Here you can look at different IceCube events and try to guess which neutrino flavor you're looking at, $\nu_e$ or $\nu_\mu$. Use the "event_no" variable to select which event to show. You can use the "reveal_network_prediction" and "reveal_true_label" variables between True and False to toggle whether to reveal the network's guess for the event and the true neutrino type of the event. The network prediction will be closer to 0 if the network thinks the event is a muon, and 1 if the network thinks the event is an electron.


In [ ]:
event_no = 1 # which event to show
ML_Helper.plot_event(event_no,
                    reveal_network_predition=False, # whether to reveal the network's guess for the event
                    reveal_true_label=False # whether to reveal the true neutrino flavor for the event
                    )

Let's see how well our network does for many events. Here we show the network many IceCube images of electrons and muons that it didn't see during training. You can see how well your network does at predicting the neutrino flavor for these events by seeing whether the muons cluster around 0 and the electrons cluster around 1 in the histogram below.

In [ ]:
# Save network scores on the test data
pred_label_test = []
true_label_test = []
for input,target in ML_Helper.test_dataloader:
    output = ML_Helper.net(input).detach().numpy()
    true_label_test += list(target[:,0])
    pred_label_test += list(output[:])
pred_test = np.array(pred_label_test,dtype=float)
true_test = np.array(true_label_test,dtype=float)

# Plot network score distributions
bins = np.linspace(0,1,10)
fig = plt.figure(figsize=(8,6))
plt.hist(pred_test[true_test==0],alpha=0.5,bins=bins,label="Muons",color="dodgerblue")
plt.hist(pred_test[true_test==1],alpha=0.5,bins=bins,label="Electrons",color="orangered")
plt.xlabel("Electron score", fontsize=14)
plt.ylabel("Number of events",fontsize=14)
plt.semilogy()
plt.legend(fontsize=14)
plt.show()

In [ ]:
print('hello')